## Se realiza el cálculo del **Beta** y la regresión lineal de **CAPM** correspondiente al ADR del banco BBVA con respecto al S&P500.

Importo las librerías

In [1]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm

Defino los símbolos de las acciones

In [2]:
symbol_bbar_adr = "BBAR" # BBAR ADR en la bolsa de Nueva York (NYSE)
symbol_snp500 = "^GSPC" # S&P 500

Defino las fechas de inicio y fin

In [3]:
start_date = "2018-05-19"
end_date = "2023-05-20"

Descargo los datos históricos de BBAR ADR y S&P500

In [4]:
bbar_adr_data = yf.download(symbol_bbar_adr, start=start_date, end=end_date)
snp500_data = yf.download(symbol_snp500, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Selecciono solo los precios de cierre ajustados

In [5]:
bbar_adr_prices = bbar_adr_data["Adj Close"]
snp500_prices = snp500_data["Adj Close"]

In [9]:
print(bbar_adr_prices.tail(n=5))

Date
2023-05-15    4.058721
2023-05-16    3.845104
2023-05-17    3.987514
2023-05-18    3.827302
2023-05-19    3.836203
Name: Adj Close, dtype: float64


Junto los precios de BBAR ADR y el S&P500 en un solo Dataframe

In [6]:
data = pd.concat([bbar_adr_prices, snp500_prices], axis=1)
data.columns = ["BBAR", "S&P500"]

Remuestreo a frecuencia mensual y calculo los rendimientos mensuales

In [7]:
monthly_data = data.resample("M").last()  #Remuestreo al último día de cada mes
monthly_returns = monthly_data.pct_change().dropna()  #Calculo los rendimientos mensuales

In [11]:
monthly_returns.tail(n=5)

,BBAR,S&P500
Date,,
2023-01-31,0.192308,0.061753
2023-02-28,0.098925,-0.026112
2023-03-31,-0.232877,0.035052
2023-04-30,0.091837,0.014642
2023-05-31,0.007009,0.005396


Calculo el Beta de bbar

In [12]:
covarianza = monthly_returns["BBAR"].cov(monthly_returns["S&P500"])
varianza_syp = monthly_returns["S&P500"].var()
beta = covarianza / varianza_syp
print("Beta = ",beta)

Beta =  1.1354753330224172


Regresión CAPM

In [13]:
# Agrego una columna de unos para el intercepto en la regresión
x = sm.add_constant(monthly_returns["S&P500"])

# Regresión Lineal
model = sm.OLS(monthly_returns["BBAR"], x)
results = model.fit()

# Imprimo los resultados
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   BBAR   R-squared:                       0.097
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                     6.251
Date:                Sun, 30 Jun 2024   Prob (F-statistic):             0.0153
Time:                        18:05:53   Log-Likelihood:                 16.323
No. Observations:                  60   AIC:                            -28.65
Df Residuals:                      58   BIC:                            -24.46
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0074      0.025     -0.301      0.7